In [57]:
! killall python

In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/
# !ps aux

In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

1kw_train_data.csv  models	       result_0629_1.csv     test_data.csv
kesci_submit	    result_0627_1.csv  result_0629_demo.csv
lost+found	    result_0627_2.csv  spark-warehouse


In [3]:
# 查看当前kernerl下的package
# !pip list --format=columns

In [4]:
# 显示cell运行时长
%load_ext klab-autotime

In [5]:
#数据查看路径如下
train_path='/home/kesci/input/bytedance/first-round/train.csv'
raw_test_path='/home/kesci/input/bytedance/first-round/test.csv'
work_path='/home/kesci/work/'
train_path_1kw = './1kw_train_data.csv'
test_path = './test_data.csv'

!pwd
!ls -lh /home/kesci/input/bytedance/first-round/
!ls -lh $train_path_1kw
!ls -lh $test_path

/home/kesci/work
total 9.0G
-rw-r--r-- 1 kesci 1000 426M May 22 04:05 test.csv
-rw-r--r-- 1 kesci 1000 8.6G May 16 20:15 train.csv
-rw-r--r-- 1 kesci users 946M Jun 20 10:25 ./1kw_train_data.csv
-rw-r--r-- 1 kesci users 463M Jun 24 15:28 ./test_data.csv
time: 1.92 s


In [6]:
import pandas as pd
import numpy as np
from math import *

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, concat, concat_ws, col, corr, avg
import pyspark.sql.functions as F

time: 717 ms


In [7]:
# # 运行一次  转换1kw数据  3min
# with open(train_path, 'r') as f:
#     # header_names=['query_id','query','title','label']
#     csv_reader = pd.read_csv(f, encoding="utf-8", engine='python',header=None,nrows=10000000)
#     csv_reader.to_csv(train_path_1kw,index=1)


# # with open(raw_test_path, 'r') as f:
# #     # header_names=['query_id','query','title','label']
# #     csv_reader = pd.read_csv(f, encoding="utf-8", engine='python',header=None)
# #     csv_reader.to_csv(test_path,index=1)
# header_names=['index','query_id','query','title','label']
# pd.read_csv(open(test_path,'r'), encoding="utf-8", engine='python',nrows=5)    

time: 642 µs


In [8]:
# all train data  
spark = SparkSession.builder.appName('ctr_pyspark_0609') \
    .config("spark.driver.memory", "29g") \
    .config("spark.executor.memory", "29g") \
    .config("spark.driver.maxResultSize","29g") \
    .config("spark.dynamicAllocation.enabled","true") \
    .config("spark.logConf", "true") \
    .getOrCreate()

    
sdf_all = spark.read.csv(train_path_1kw, header=False) \
    .toDF("index", "query_id", "query", "query_title_id", "title", 'label')
sdf_all = sdf_all.filter(sdf_all.index >= 5000000) \
    .withColumn("index", sdf_all.index.cast("Int")) \
    .withColumn("label", sdf_all.label.cast("Int")) \
    .withColumn("query_id", sdf_all.query_id.cast("Int")) \
    .withColumn("query_title_id", sdf_all.query_title_id.cast("Int")) \
    .withColumn("text", concat_ws(' ', sdf_all.query, sdf_all.title))
# 
all_count = sdf_all.count()
print(sdf_all.dtypes,all_count)
# sdf_all.show(2,truncate=True)

[('index', 'int'), ('query_id', 'int'), ('query', 'string'), ('query_title_id', 'int'), ('title', 'string'), ('label', 'int'), ('text', 'string')] 5000000
time: 22.1 s


In [9]:
# !spark-shell  --help

time: 564 µs


In [10]:
# test data
sdf_test = spark.read.csv(test_path,header=True) \
    .toDF("index","query_id", "query", "query_title_id", "title")
sdf_test = sdf_test.withColumn("index", sdf_test.index.cast("Int")) \
    .withColumn("text", concat_ws(' ',sdf_test.query,sdf_test.title)) \
    .withColumn("query_id", sdf_test.query_id.cast("Int")) \
    .withColumn("query_title_id", sdf_test.query_title_id.cast("Int")) \
    # .filter(sdf_test.index < 100)

print(sdf_test.dtypes)
# sdf_test.show(2,truncate=True)

[('index', 'int'), ('query_id', 'int'), ('query', 'string'), ('query_title_id', 'int'), ('title', 'string'), ('text', 'string')]
time: 632 ms


In [11]:
# reset datasets
# sdf_all = sdf_all.select("index", "query_id", "query", "query_title_id", "title","text","label")
# sdf_test = sdf_test.select("index","query_id", "query", "query_title_id", "title","text")

time: 433 µs


In [12]:
from pyspark.ml.feature import Normalizer,MinHashLSH,HashingTF,IDF,IDFModel,Tokenizer,Word2Vec, Word2VecModel,IndexToString, StringIndexer, VectorIndexer

# tokenizer 
tokenizer = Tokenizer(inputCol='text', outputCol='texts')
sdf_all = tokenizer.transform(sdf_all)
sdf_all = tokenizer.transform(sdf_all,params={tokenizer.inputCol:"query",tokenizer.outputCol:"queries"})
sdf_all = tokenizer.transform(sdf_all,params={tokenizer.inputCol:"title",tokenizer.outputCol:"titles"})
sdf_test = tokenizer.transform(sdf_test)
sdf_test = tokenizer.transform(sdf_test,params={tokenizer.inputCol:"query",tokenizer.outputCol:"queries"})
sdf_test = tokenizer.transform(sdf_test,params={tokenizer.inputCol:"title",tokenizer.outputCol:"titles"})

print(sdf_all.dtypes)
print(sdf_test.dtypes)


[('index', 'int'), ('query_id', 'int'), ('query', 'string'), ('query_title_id', 'int'), ('title', 'string'), ('label', 'int'), ('text', 'string'), ('texts', 'array<string>'), ('queries', 'array<string>'), ('titles', 'array<string>')]
[('index', 'int'), ('query_id', 'int'), ('query', 'string'), ('query_title_id', 'int'), ('title', 'string'), ('text', 'string'), ('texts', 'array<string>'), ('queries', 'array<string>'), ('titles', 'array<string>')]
time: 299 ms


In [13]:
# word-vector

word2vecModelPath =  "./models/word2vec-model-1kw-50-1-2" 
idfModelPath =  "./models/idf-model-1kw-100000-2" 

# word2vec
# word2vec = Word2Vec(inputCol="texts",outputCol='word2vec',numPartitions=64,vectorSize=50,maxIter=1,minCount=2,seed=2019)
# print("word2vec parameters:\n" + word2vec.explainParams())
# word2vecModel = word2vec.fit(sdf_test.select("texts").unionByName(sdf_all.select("texts")))
# word2vecModel.save(word2vecModelPath)

# idf
# tf = HashingTF(inputCol="texts", outputCol="tf", numFeatures=100000)
# idf = IDF(inputCol="tf", outputCol="idf",minDocFreq=2)
# print("tf parameters:\n" + tf.explainParams())
# print("idf parameters:\n" + idf.explainParams())
# idfModel = idf.fit(tf.transform(sdf_test).select("tf").unionByName(tf.transform(sdf_all).select("tf")))
# idfModel.save(idfModelPath)

# idfModel.save(idfModelPath)

In [14]:
# word2vec
word2vecModel = Word2VecModel.load(word2vecModelPath)
sdf_all = word2vecModel.transform(sdf_all,params={word2vecModel.inputCol:"titles",word2vecModel.outputCol:"word2vec_title"})
sdf_all = word2vecModel.transform(sdf_all,params={word2vecModel.inputCol:"queries",word2vecModel.outputCol:"word2vec_query"})
sdf_test = word2vecModel.transform(sdf_test,params={word2vecModel.inputCol:"titles",word2vecModel.outputCol:"word2vec_title"})
sdf_test = word2vecModel.transform(sdf_test,params={word2vecModel.inputCol:"queries",word2vecModel.outputCol:"word2vec_query"})

# tfidf
tf = HashingTF(inputCol="texts", outputCol="tf", numFeatures=100000)
idfModel = IDFModel.load(idfModelPath)
normalizer = Normalizer(inputCol="idf", outputCol="norm_idf")

sdf_all = tf.transform(sdf_all,params={tf.inputCol:"titles",tf.outputCol:"tf_title"})
sdf_all = tf.transform(sdf_all,params={tf.inputCol:"queries",tf.outputCol:"tf_query"})
sdf_test = tf.transform(sdf_test,params={tf.inputCol:"titles",tf.outputCol:"tf_title"})
sdf_test = tf.transform(sdf_test,params={tf.inputCol:"queries",tf.outputCol:"tf_query"})

sdf_all = idfModel.transform(sdf_all,params={idfModel.inputCol:"tf_title",idfModel.outputCol:"idf_title"})
sdf_all = idfModel.transform(sdf_all,params={idfModel.inputCol:"tf_query",idfModel.outputCol:"idf_query"})
sdf_test = idfModel.transform(sdf_test,params={idfModel.inputCol:"tf_title",idfModel.outputCol:"idf_title"})
sdf_test = idfModel.transform(sdf_test,params={idfModel.inputCol:"tf_query",idfModel.outputCol:"idf_query"})

sdf_all = normalizer.transform(sdf_all,params={normalizer.inputCol:"idf_title",normalizer.outputCol:"norm_idf_title"})
sdf_all = normalizer.transform(sdf_all,params={normalizer.inputCol:"idf_query",normalizer.outputCol:"norm_idf_query"})
sdf_test = normalizer.transform(sdf_test,params={normalizer.inputCol:"idf_title",normalizer.outputCol:"norm_idf_title"})
sdf_test = normalizer.transform(sdf_test,params={normalizer.inputCol:"idf_query",normalizer.outputCol:"norm_idf_query"})



In [17]:
sdf_all.dtypes

In [16]:
from pyspark.ml.feature import NGram

biGram = NGram(n=2, inputCol="texts", outputCol="bi_texts") # ngram.setParams(n=4)

sdf_all = biGram.transform(sdf_all)
sdf_all = biGram.transform(sdf_all,params={biGram.inputCol:"queries",biGram.outputCol:"bi_queries"})
sdf_all = biGram.transform(sdf_all,params={biGram.inputCol:"titles",biGram.outputCol:"bi_titles"})
sdf_test = biGram.transform(sdf_test)
sdf_test = biGram.transform(sdf_test,params={biGram.inputCol:"queries",biGram.outputCol:"bi_queries"})
sdf_test = biGram.transform(sdf_test,params={biGram.inputCol:"titles",biGram.outputCol:"bi_titles"})

#bi

biWord2vecModelPath =  "./models/bi-word2vec-model-1kw-50-1-2" 
biIdfModelPath =  "./models/bi-idf-model-1kw-100000-2" 
# word2vec
word2vec = Word2Vec(inputCol="bi_texts",outputCol='bi_word2vec',numPartitions=64,vectorSize=50,maxIter=1,minCount=2,seed=2019)
word2vecModel = word2vec.fit(sdf_test.select("bi_texts").unionByName(sdf_all.select("bi_texts")))
word2vecModel.save(biWord2vecModelPath)

# # idf
tf = HashingTF(inputCol="bi_texts", outputCol="bi_tf", numFeatures=100000)
idf = IDF(inputCol="bi_tf", outputCol="bi_idf",minDocFreq=2)
idfModel = idf.fit(tf.transform(sdf_test).select("bi_tf").unionByName(tf.transform(sdf_all).select("bi_tf")))
idfModel.save(biIdfModelPath)


In [17]:
# bi
# word2vec
biWord2vecModel = Word2VecModel.load(biWord2vecModelPath)
sdf_all = biWord2vecModel.transform(sdf_all,params={biWord2vecModel.inputCol:"bi_titles",biWord2vecModel.outputCol:"bi_word2vec_title"})
sdf_all = biWord2vecModel.transform(sdf_all,params={biWord2vecModel.inputCol:"bi_queries",biWord2vecModel.outputCol:"bi_word2vec_query"})
sdf_test = biWord2vecModel.transform(sdf_test,params={biWord2vecModel.inputCol:"bi_titles",biWord2vecModel.outputCol:"bi_word2vec_title"})
sdf_test = biWord2vecModel.transform(sdf_test,params={biWord2vecModel.inputCol:"bi_queries",biWord2vecModel.outputCol:"bi_word2vec_query"})

# tfidf
biTf = HashingTF(inputCol="bi_texts", outputCol="bi_tf", numFeatures=10)
biIdfModel = IDFModel.load(biIdfModelPath)
biNormalizer = Normalizer(inputCol="bi_idf", outputCol="bi_norm_idf")

sdf_all = biTf.transform(sdf_all,params={biTf.inputCol:"bi_titles",biTf.outputCol:"bi_tf_title"})
sdf_all = biTf.transform(sdf_all,params={biTf.inputCol:"bi_queries",biTf.outputCol:"bi_tf_query"})
sdf_test = biTf.transform(sdf_test,params={biTf.inputCol:"bi_titles",biTf.outputCol:"bi_tf_title"})
sdf_test = biTf.transform(sdf_test,params={biTf.inputCol:"bi_queries",biTf.outputCol:"bi_tf_query"})

sdf_all = biIdfModel.transform(sdf_all,params={biIdfModel.inputCol:"bi_tf_title",biIdfModel.outputCol:"bi_idf_title"})
sdf_all = biIdfModel.transform(sdf_all,params={biIdfModel.inputCol:"bi_tf_query",biIdfModel.outputCol:"bi_idf_query"})
sdf_test = biIdfModel.transform(sdf_test,params={biIdfModel.inputCol:"bi_tf_title",biIdfModel.outputCol:"bi_idf_title"})
sdf_test = biIdfModel.transform(sdf_test,params={biIdfModel.inputCol:"bi_tf_query",biIdfModel.outputCol:"bi_idf_query"})

sdf_all = biNormalizer.transform(sdf_all,params={biNormalizer.inputCol:"bi_idf_title",biNormalizer.outputCol:"bi_norm_idf_title"})
sdf_all = biNormalizer.transform(sdf_all,params={biNormalizer.inputCol:"bi_idf_query",biNormalizer.outputCol:"bi_norm_idf_query"})
sdf_test = biNormalizer.transform(sdf_test,params={biNormalizer.inputCol:"bi_idf_title",biNormalizer.outputCol:"bi_norm_idf_title"})
sdf_test = biNormalizer.transform(sdf_test,params={biNormalizer.inputCol:"bi_idf_query",biNormalizer.outputCol:"bi_norm_idf_query"})



In [18]:
sdf_all.dtypes
# sdf_all.select('title').groupBy('title').count().filter(col('count')>50).orderBy(col("count").desc()).collect()

In [19]:
import numpy as np
import time
from pyspark.sql import SQLContext
from pyspark.sql import Row
import Levenshtein

def dist(A,B,mode):
        
    if mode == "euclidean":
        sim = np.linalg.norm(A - B)  # euclidean
    elif mode == "cosine":
        num = np.sum(np.multiply(A,B))
        denom = np.linalg.norm(A) * np.linalg.norm(B)
        if denom == 0:
            denom = 1e-6   # 除0处理
        sim = np.divide(num, denom)  # 0.5 + 0.5 * cos
    elif  mode == "edit":
        sim  = Levenshtein.distance(A,B)
    return round(float(sim),6)
  
def square_rooted(x):
    return sqrt(sum([a*a for a in x]))
    
def cosine_sim(x,y):
    x,y = [float(i) for i in x],[float(j) for j in y]
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(float(np.divide(numerator,denominator+1e-6)),6)

def jaccard_sim(x,y):
    x, y = set(x), set(y)
    inter = len(set.intersection(*[x, y]))
    union = len(set.union(*[x, y]))
    return round(float(np.divide(inter,union)),6)

def sim(x, y, mode="cosine"):
    if mode == "jaccard":
        d = jaccard_sim(x, y)
    elif mode == "cosine":
        d = cosine_sim(x, y)
    return round(float(d),6)

def title_len(x):
    return len(x["titles"])

def word_count(x):
    return sum(1 for i in x['titles'] if i in x['queries'])


def get_pos(query, text):
    return [round(float( np.divide(len(text),(1+text.index(i))) ),6)  if i in set(text) else 0. for i in set(query)]
    
def mean_pos(x):
    return round(float(np.mean(get_pos(x['queries'], x['titles']))),6)

def word_ratio(x):
    return round(float(np.divide(len(set(x['queries'])),len(set(x['titles'])))),6)

def get_qt(query, title):
    # query在title中的位置  平均值
    return [title.index(i)  if i in set(title) else len(title) for i in set(query)]
def get_qt_min(query, title):
    # query在text中的位置  max
    tmp = get_qt(query, title)
    return int(tmp[np.argmin(tmp)])
def get_qt_mean(query, title):
    # query在text中的位置  max
    tmp = get_qt(query, title)
    return round(float(np.mean(tmp)),6)
# 
def sum_hit(x,y):
    # print('sum_hit')
    return sum(x.count(i) for i in set(y))
def mean_hit(x,y):
    # print('mean_hit')
    return  round(float(np.mean([x.count(i) for i in set(y)])),6)
def max_hit(x,y):
    # print('max_hit')
    return max(x.count(i) for i in set(y) )
    

In [20]:
row = Row("index","w2v_eu_dist","w2v_cos_dist","edit_dist",
    "text_cos_sim","text_jaccard_sim","idf_cos_sim",
    "title_len",'word_count','mean_pos','word_ratio',
    "qt_min","qt_mean","tq_min","tq_mean",
    "sum_hit","mean_hit","max_hit",
    "bi_1","bi_2","bi_3","bi_4","bi_5","bi_6","bi_7","bi_8","bi_9","bi_10","bi_11","bi_12","bi_13","bi_14","bi_15")    
#  title_len 
fea_all = sdf_all.select(
        "index","title","query","titles","queries","word2vec_title","word2vec_query","norm_idf_title","norm_idf_query", \
         "bi_titles","bi_queries","bi_word2vec_title","bi_word2vec_query","bi_norm_idf_title","bi_norm_idf_query"
    ).rdd.map(lambda x:row(
            x["index"],
            dist(x["word2vec_title"],x['word2vec_query'],mode="euclidean"), 
            dist(x["word2vec_title"],x['word2vec_query'],mode="cosine"),
            dist(x['title'], x['query'],mode="edit"),
            sim(x['titles'], x['queries'],mode="cosine"),
            sim(x['titles'], x['queries'],mode="jaccard"),
            sim(x['norm_idf_title'].toArray(), x['norm_idf_query'].toArray(),mode="cosine"),
            title_len(x),
            word_count(x),
            mean_pos(x),
            word_ratio(x),
            get_qt_min(x["queries"],x["titles"]),
            get_qt_mean(x["queries"],x["titles"]),
            get_qt_min(x["titles"],x["queries"]),
            get_qt_mean(x["titles"],x["queries"]),
            sum_hit(x['titles'],x['queries']),
            mean_hit(x['titles'],x['queries']),
            max_hit(x['titles'],x['queries']), \
            dist(x["bi_word2vec_title"],x['bi_word2vec_query'],mode="euclidean"), 
            dist(x["bi_word2vec_title"],x['bi_word2vec_query'],mode="cosine"),
            sim(x['bi_titles'], x['queries'],mode="jaccard"),
            sim(x['bi_norm_idf_title'].toArray(), x['bi_norm_idf_query'].toArray(),mode="cosine"),
            len(x["bi_titles"]),
            sum(1 for i in x['bi_titles'] if i in x['bi_queries']),
            round(float(np.mean(get_pos(x['bi_queries'], x['bi_titles']))),6),
            round(float(np.divide(len(set(x['bi_queries'])),len(set(x['bi_titles'])))),6),
            get_qt_min(x["bi_queries"],x["bi_titles"]),
            get_qt_mean(x["bi_queries"],x["bi_titles"]),
            get_qt_min(x["bi_titles"],x["bi_queries"]),
            get_qt_mean(x["bi_titles"],x["bi_queries"]),
            sum_hit(x['titles'],x['queries']),
            mean_hit(x['titles'],x['queries']),
            max_hit(x['titles'],x['queries'])
        )
    ).toDF().join(sdf_all.select('index','query_id','query_title_id','title','label'), on='index')

# test
fea_test = sdf_test.select( 
        "index","title","query","titles","queries","word2vec_title", "word2vec_query","norm_idf_title","norm_idf_query", \
         "bi_titles","bi_queries","bi_word2vec_title","bi_word2vec_query","bi_norm_idf_title","bi_norm_idf_query"
    ).rdd.map(lambda x:row(
            x["index"],
            dist(x["word2vec_title"],x['word2vec_query'],mode="euclidean"), 
            dist(x["word2vec_title"],x['word2vec_query'],mode="cosine"),
            dist(x['title'], x['query'],mode="edit"),
            sim(x['titles'], x['queries'],mode="cosine"),
            sim(x['titles'], x['queries'],mode="jaccard"),
            sim(x['norm_idf_title'].toArray(), x['norm_idf_query'].toArray(),mode="cosine"),
            title_len(x),
            word_count(x),
            mean_pos(x),
            word_ratio(x),
            get_qt_min(x["queries"],x["titles"]),
            get_qt_mean(x["queries"],x["titles"]),
            get_qt_min(x["titles"],x["queries"]),
            get_qt_mean(x["titles"],x["queries"]),
            sum_hit(x['titles'],x['queries']),
            mean_hit(x['titles'],x['queries']),
            max_hit(x['titles'],x['queries']), \
            dist(x["bi_word2vec_title"],x['bi_word2vec_query'],mode="euclidean"), 
            dist(x["bi_word2vec_title"],x['bi_word2vec_query'],mode="cosine"),
            sim(x['bi_titles'], x['queries'],mode="jaccard"),
            sim(x['bi_norm_idf_title'].toArray(), x['bi_norm_idf_query'].toArray(),mode="cosine"),
            len(x["bi_titles"]),
            sum(1 for i in x['bi_titles'] if i in x['bi_queries']),
            round(float(np.mean(get_pos(x['bi_queries'], x['bi_titles']))),6),
            round(float(np.divide(len(set(x['bi_queries'])),len(set(x['bi_titles'])))),6),
            get_qt_min(x["bi_queries"],x["bi_titles"]),
            get_qt_mean(x["bi_queries"],x["bi_titles"]),
            get_qt_min(x["bi_titles"],x["bi_queries"]),
            get_qt_mean(x["bi_titles"],x["bi_queries"]),
            sum_hit(x['bi_titles'],x['bi_queries']),
            mean_hit(x['bi_titles'],x['bi_queries']),
            max_hit(x['bi_titles'],x['bi_queries'])
        ) 
    ).toDF().join(sdf_test.select('index','query_id','query_title_id','title') , on='index')


In [43]:
fea_all.dtypes

In [65]:

fea_all.join(fea_all.groupby('query_id').agg(F.max(fea_all.w2v_eu_dist),F.min(fea_all.w2v_eu_dist),
                                F.max(fea_all.w2v_cos_dist),F.min(fea_all.w2v_cos_dist),
                                F.max(fea_all.edit_dist),F.min(fea_all.edit_dist),
                                F.max(fea_all.text_cos_sim),F.min(fea_all.text_cos_sim),
                                F.max(fea_all.text_jaccard_sim),F.min(fea_all.text_jaccard_sim),
                                F.max(fea_all.idf_cos_sim),F.min(fea_all.idf_cos_sim),
                                F.max(fea_all.title_len),F.min(fea_all.title_len),
                                F.max(fea_all.word_count),F.min(fea_all.word_count),
                                F.max(fea_all.mean_pos),F.min(fea_all.mean_pos),
                                F.max(fea_all.word_ratio),F.min(fea_all.word_ratio),
                                F.max(fea_all.qt_min),F.min(fea_all.qt_min),
                                F.max(fea_all.qt_mean),F.min(fea_all.qt_mean),
                                F.max(fea_all.tq_min),F.min(fea_all.tq_min),
                                F.max(fea_all.tq_mean),F.min(fea_all.tq_mean),
                                F.max(fea_all.sum_hit),F.min(fea_all.sum_hit),
                                F.max(fea_all.mean_hit),F.min(fea_all.mean_hit),
                                F.max(fea_all.max_hit),F.min(fea_all.max_hit),
                                F.max(fea_all.bi_1),F.min(fea_all.bi_1),
                                F.max(fea_all.bi_2),F.min(fea_all.bi_2),
                                F.max(fea_all.bi_3),F.min(fea_all.bi_3),
                                F.max(fea_all.bi_4),F.min(fea_all.bi_4),
                                F.max(fea_all.bi_5),F.min(fea_all.bi_5),
                                F.max(fea_all.bi_6),F.min(fea_all.bi_6),
                                F.max(fea_all.bi_7),F.min(fea_all.bi_7),
                                F.max(fea_all.bi_8),F.min(fea_all.bi_8),
                                F.max(fea_all.bi_9),F.min(fea_all.bi_9),
                                F.max(fea_all.bi_10),F.min(fea_all.bi_10),
                                F.max(fea_all.bi_11),F.min(fea_all.bi_11),
                                F.max(fea_all.bi_12),F.min(fea_all.bi_12),
                                F.max(fea_all.bi_13),F.min(fea_all.bi_13),
                                F.max(fea_all.bi_14),F.min(fea_all.bi_14),
                                F.max(fea_all.bi_15),F.min(fea_all.bi_15),
                                F.count(fea_all.query_id),
                                F.avg(fea_all.title_len)),
                            on='query_id').orderBy('index').rdd.map(lambda x:row(
                                    x["index"],
                                    norm_fea(x,'w2v_eu_dist'),
                                    norm_fea(x,'w2v_cos_dist'),
                                    norm_fea(x,'edit_dist'),
                                    norm_fea(x,'text_cos_sim'),
                                    norm_fea(x,'text_jaccard_sim'),
                                    norm_fea(x,'idf_cos_sim'),
                                    norm_fea(x,'title_len'),
                                    norm_fea(x,'word_count'),
                                    norm_fea(x,'mean_pos'),
                                    norm_fea(x,'word_ratio'),
                                    norm_fea(x,'qt_min'),
                                    norm_fea(x,'qt_mean'),
                                    norm_fea(x,'tq_min'),
                                    norm_fea(x,'tq_mean'),
                                    norm_fea(x,'sum_hit'),
                                    norm_fea(x,'mean_hit'),
                                    norm_fea(x,'max_hit'),
                                    norm_fea(x,'bi_1'),
                                    norm_fea(x,'bi_2'),
                                    norm_fea(x,'bi_3'),
                                    norm_fea(x,'bi_4'),
                                    norm_fea(x,'bi_5'),
                                    norm_fea(x,'bi_6'),
                                    norm_fea(x,'bi_7'),
                                    norm_fea(x,'bi_8'),
                                    norm_fea(x,'bi_9'),
                                    norm_fea(x,'bi_10'),
                                    norm_fea(x,'bi_11'),
                                    norm_fea(x,'bi_12'),
                                    norm_fea(x,'bi_13'),
                                    norm_fea(x,'bi_14'),
                                    norm_fea(x,'bi_15')
                                    )).take(1)
                                    # .toDF()
                                    # .join(sdf_all.select('index','query_id','query_title_id','title','label'), on='index')

In [67]:
##

In [68]:
def norm_fea(x,s):
    max_str = 'max('+s+')'
    min_str = 'min('+s+')'
    return round(float(np.divide(x[s]-x[min_str]+ 1e-6,x[max_str]-x[min_str]+ 1e-6)),6)

sdf_all =  fea_all.join(sdf_all.groupby('title').count().withColumnRenamed("count", "t_retri_count"), on='title') \
    .join(fea_all.groupby('query_id').agg(
            F.max(fea_all.w2v_eu_dist),F.min(fea_all.w2v_eu_dist),
            F.max(fea_all.w2v_cos_dist),F.min(fea_all.w2v_cos_dist),
            F.max(fea_all.edit_dist),F.min(fea_all.edit_dist),
            F.max(fea_all.text_cos_sim),F.min(fea_all.text_cos_sim),
            F.max(fea_all.text_jaccard_sim),F.min(fea_all.text_jaccard_sim),
            F.max(fea_all.idf_cos_sim),F.min(fea_all.idf_cos_sim),
            F.max(fea_all.title_len),F.min(fea_all.title_len),
            F.max(fea_all.word_count),F.min(fea_all.word_count),
            F.max(fea_all.mean_pos),F.min(fea_all.mean_pos),
            F.max(fea_all.word_ratio),F.min(fea_all.word_ratio),
            F.max(fea_all.qt_min),F.min(fea_all.qt_min),
            F.max(fea_all.qt_mean),F.min(fea_all.qt_mean),
            F.max(fea_all.tq_min),F.min(fea_all.tq_min),
            F.max(fea_all.tq_mean),F.min(fea_all.tq_mean),
            F.max(fea_all.sum_hit),F.min(fea_all.sum_hit),
            F.max(fea_all.mean_hit),F.min(fea_all.mean_hit),
            F.max(fea_all.max_hit),F.min(fea_all.max_hit),
            F.max(fea_all.bi_1),F.min(fea_all.bi_1),
            F.max(fea_all.bi_2),F.min(fea_all.bi_2),
            F.max(fea_all.bi_3),F.min(fea_all.bi_3),
            F.max(fea_all.bi_4),F.min(fea_all.bi_4),
            F.max(fea_all.bi_5),F.min(fea_all.bi_5),
            F.max(fea_all.bi_6),F.min(fea_all.bi_6),
            F.max(fea_all.bi_7),F.min(fea_all.bi_7),
            F.max(fea_all.bi_8),F.min(fea_all.bi_8),
            F.max(fea_all.bi_9),F.min(fea_all.bi_9),
            F.max(fea_all.bi_10),F.min(fea_all.bi_10),
            F.max(fea_all.bi_11),F.min(fea_all.bi_11),
            F.max(fea_all.bi_12),F.min(fea_all.bi_12),
            F.max(fea_all.bi_13),F.min(fea_all.bi_13),
            F.max(fea_all.bi_14),F.min(fea_all.bi_14),
            F.max(fea_all.bi_15),F.min(fea_all.bi_15),
            F.count(fea_all.query_id),
            F.avg(fea_all.title_len)
        ), on='query_id'
    ).rdd.map(lambda x:row(
            x["index"],
            norm_fea(x,'w2v_eu_dist'),
            norm_fea(x,'w2v_cos_dist'),
            norm_fea(x,'edit_dist'),
            norm_fea(x,'text_cos_sim'),
            norm_fea(x,'text_jaccard_sim'),
            norm_fea(x,'idf_cos_sim'),
            norm_fea(x,'title_len'),
            norm_fea(x,'word_count'),
            norm_fea(x,'mean_pos'),
            norm_fea(x,'word_ratio'),
            norm_fea(x,'qt_min'),
            norm_fea(x,'qt_mean'),
            norm_fea(x,'tq_min'),
            norm_fea(x,'tq_mean'),
            norm_fea(x,'sum_hit'),
            norm_fea(x,'mean_hit'),
            norm_fea(x,'max_hit'),
            norm_fea(x,'bi_1'),
            norm_fea(x,'bi_2'),
            norm_fea(x,'bi_3'),
            norm_fea(x,'bi_4'),
            norm_fea(x,'bi_5'),
            norm_fea(x,'bi_6'),
            norm_fea(x,'bi_7'),
            norm_fea(x,'bi_8'),
            norm_fea(x,'bi_9'),
            norm_fea(x,'bi_10'),
            norm_fea(x,'bi_11'),
            norm_fea(x,'bi_12'),
            norm_fea(x,'bi_13'),
            norm_fea(x,'bi_14'),
            norm_fea(x,'bi_15')
        )
    ).toDF().join(sdf_all.select('index','query_id','query_title_id','title','label'), on='index').orderBy('index')
                                        
    
sdf_test = fea_test.join(sdf_test.groupby('title').count().withColumnRenamed("count", "t_retri_count"), on='title') \
    .join(fea_test.groupby('query_id').agg(
            F.max(fea_test.w2v_eu_dist),F.min(fea_test.w2v_eu_dist),
            F.max(fea_test.w2v_cos_dist),F.min(fea_test.w2v_cos_dist),
            F.max(fea_test.edit_dist),F.min(fea_test.edit_dist),
            F.max(fea_test.text_cos_sim),F.min(fea_test.text_cos_sim),
            F.max(fea_test.text_jaccard_sim),F.min(fea_test.text_jaccard_sim),
            F.max(fea_test.idf_cos_sim),F.min(fea_test.idf_cos_sim),
            F.max(fea_test.title_len),F.min(fea_test.title_len),
            F.max(fea_test.word_count),F.min(fea_test.word_count),
            F.max(fea_test.mean_pos),F.min(fea_test.mean_pos),
            F.max(fea_test.word_ratio),F.min(fea_test.word_ratio),
            F.max(fea_test.qt_min),F.min(fea_test.qt_min),
            F.max(fea_test.qt_mean),F.min(fea_test.qt_mean),
            F.max(fea_test.tq_min),F.min(fea_test.tq_min),
            F.max(fea_test.tq_mean),F.min(fea_test.tq_mean),
            F.max(fea_test.sum_hit),F.min(fea_test.sum_hit),
            F.max(fea_test.mean_hit),F.min(fea_test.mean_hit),
            F.max(fea_test.max_hit),F.min(fea_test.max_hit),
            F.max(fea_test.bi_1),F.min(fea_test.bi_1),
            F.max(fea_test.bi_2),F.min(fea_test.bi_2),
            F.max(fea_test.bi_3),F.min(fea_test.bi_3),
            F.max(fea_test.bi_4),F.min(fea_test.bi_4),
            F.max(fea_test.bi_5),F.min(fea_test.bi_5),
            F.max(fea_test.bi_6),F.min(fea_test.bi_6),
            F.max(fea_test.bi_7),F.min(fea_test.bi_7),
            F.max(fea_test.bi_8),F.min(fea_test.bi_8),
            F.max(fea_test.bi_9),F.min(fea_test.bi_9),
            F.max(fea_test.bi_10),F.min(fea_test.bi_10),
            F.max(fea_test.bi_11),F.min(fea_test.bi_11),
            F.max(fea_test.bi_12),F.min(fea_test.bi_12),
            F.max(fea_test.bi_13),F.min(fea_test.bi_13),
            F.max(fea_test.bi_14),F.min(fea_test.bi_14),
            F.max(fea_test.bi_15),F.min(fea_test.bi_15),
            F.count(fea_test.query_id),
            F.avg(fea_test.title_len)
        ), on='query_id'
    ).rdd.map(lambda x:row(
            x["index"],
            norm_fea(x,'w2v_eu_dist'),
            norm_fea(x,'w2v_cos_dist'),
            norm_fea(x,'edit_dist'),
            norm_fea(x,'text_cos_sim'),
            norm_fea(x,'text_jaccard_sim'),
            norm_fea(x,'idf_cos_sim'),
            norm_fea(x,'title_len'),
            norm_fea(x,'word_count'),
            norm_fea(x,'mean_pos'),
            norm_fea(x,'word_ratio'),
            norm_fea(x,'qt_min'),
            norm_fea(x,'qt_mean'),
            norm_fea(x,'tq_min'),
            norm_fea(x,'tq_mean'),
            norm_fea(x,'sum_hit'),
            norm_fea(x,'mean_hit'),
            norm_fea(x,'max_hit'),
            norm_fea(x,'bi_1'),
            norm_fea(x,'bi_2'),
            norm_fea(x,'bi_3'),
            norm_fea(x,'bi_4'),
            norm_fea(x,'bi_5'),
            norm_fea(x,'bi_6'),
            norm_fea(x,'bi_7'),
            norm_fea(x,'bi_8'),
            norm_fea(x,'bi_9'),
            norm_fea(x,'bi_10'),
            norm_fea(x,'bi_11'),
            norm_fea(x,'bi_12'),
            norm_fea(x,'bi_13'),
            norm_fea(x,'bi_14'),
            norm_fea(x,'bi_15')
        )
    ).toDF().join(sdf_test.select('index','query_id','query_title_id','title'), on='index').orderBy('index')
                                        
# sdf_all =  fea_all.join(sdf_all.groupby('title').count().withColumnRenamed("count", "t_retri_count"), on='title') \
#     .join(fea_all.groupBy('query_id').agg({'query_id':'count','title_len':'avg'}), on='query_id') \
#     .orderBy('index')

# sdf_test = fea_test.join(sdf_test.groupby('title').count().withColumnRenamed("count", "t_retri_count"), on='title') \
#     .join(fea_test.groupBy('query_id').agg({'query_id':'count','title_len':'avg'}), on='query_id') \
#     .orderBy('index') 

print(sdf_all.dtypes)
print(sdf_test.dtypes)                                        
                                        

In [1]:
sdf_all.dtypes

In [25]:
# sdf_train = sdf_all

s_len = all_count*0.9 + 9000000 
sdf_dev = sdf_all.filter(sdf_all.index>=s_len)
sdf_train = sdf_all.filter(sdf_all.index<s_len) # sdf_all.subtract(sdf_dev)


In [24]:
# model training 100w 16min 22min  4.5h
from pyspark.ml.classification import LinearSVC,GBTClassifier,LogisticRegression,LogisticRegressionModel,NaiveBayes, NaiveBayesModel

from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline,PipelineModel

 
#vector assembler
vectorAssembler = VectorAssembler(
    inputCols=[
        "w2v_eu_dist","w2v_cos_dist","edit_dist",
        "text_cos_sim","text_jaccard_sim",#"idf_cos_sim",
        "title_len",'word_count','mean_pos','word_ratio',
        "qt_min","qt_mean","tq_min","tq_mean",
        't_retri_count','count(query_id)',"avg(title_len)"],
    outputCol="assemblerFeatures")

lr = LogisticRegression(maxIter=10,regParam=0.0,elasticNetParam=0.0).setLabelCol("label").setFeaturesCol("assemblerFeatures")
nb = NaiveBayes(smoothing=1.0).setLabelCol("label").setFeaturesCol("assemblerFeatures")
# svm = LinearSVC(maxIter=5, regParam=0.01).setLabelCol("label").setFeaturesCol("assemblerFeatures")
gbt = GBTClassifier(maxDepth=5,maxIter=1,maxMemoryInMB=8192,cacheNodeIds=True).setLabelCol("label").setFeaturesCol("assemblerFeatures") # 40min
print("model parameters:\n" + lr.explainParams())

pipeline =  Pipeline().setStages([vectorAssembler,lr])
sdf_train.persist()
pipelineModel = pipeline.fit(sdf_train)
sdf_train.unpersist()



In [23]:
# model save & load
pipelineModelPath = './models/svm_model_0629_2' 
pipelineModel.save(pipelineModelPath)
# pipelineModel = PipelineModel.load(pipelineModelPath)

In [26]:
# result = pipelineModel.transform(sdf_test)
# print(result.dtypes)  

# result.select('rawPrediction','prediction').filter(col('query_id')<50250).take(10)

In [28]:
# result generate 10min 23min  4h 6.5h
result_row = Row('query_id','query_title_id','prediction')
result = pipelineModel.transform(sdf_test) \
    .select('query_id','query_title_id','probability') \
    .rdd.map(lambda x:result_row(x["query_id"],x["query_title_id"],float(x["probability"][1]))).toDF() 
    
print(result.dtypes)    

In [29]:
# result save
result.toPandas().to_csv('./result_0629_demo.csv',header=False,index=False) 

In [30]:
# result lookup
# pd.read_csv('./result_0625_7.csv',nrows=10).head()
sdf_sub = spark.read.csv('./result_0629_demo.csv')
print(sdf_sub.dtypes)
sdf_sub.show(2,truncate=True)


In [25]:
# result submit
# 500w-> 0.5326 100w -> 0.5302
# df729382a67a8015
!./kesci_submit -token b321e54849cd528c -file ./result_0627_2.csv

In [33]:
sdf_dev.dtypes
# sdf_dev.show(1,truncate=True)

In [30]:
# dev-result generate 5min
dev_result_row = Row('query_id','label','prediction')
dev_result = pipelineModel.transform(sdf_dev) \
    .select('query_id','label','probability') \
    .rdd.map(lambda x:dev_result_row(x["query_id"],x["label"],float(x["probability"][1]))).toDF() \

print(dev_result.dtypes)

In [31]:
# dev-result valid  40s
from sklearn.metrics import accuracy_score,roc_auc_score

def q_auc(df):
    invalid_count = 0
    query_ids = df['query_id'].unique()
    aucs = []
    for i,query_id in enumerate(query_ids):
        current= df[df['query_id'] == query_id]
        if sum(current['label']) not in [0, len(current['label'])]:
            auc = roc_auc_score(current['label'], current['prediction'])  # 这里计算qauc
        else:
            auc = 0.5
            invalid_count += 1
        aucs.append(auc)
    print(f'q_auc invalid_count = {invalid_count}')
    return np.mean(aucs)
    
q_auc = q_auc(dev_result.toPandas())

print(q_auc) 

offline  | model   |   tip    
-------- | -------- | -------- 
0.525 |   lr   |  100w,8-feat,0.9 
0.527	|		lr	|  drop('eu_dist')
0.528 | lr  | word2vec-model-6kw-50-5, edit_dist
0.529 | lr  | word2vec-model-6kw-50-5-2, edit_dist
0.530 | lr  | word2vec-model-6kw-50-5-2, edit_dist, text_cos_sim
0.529-0.537 | lr  | tfidf
-0.539 | lr | tfidf+norm

In [3]:
def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]
def to_pandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    
    add_label = False  if 'label' not in df.columns else True
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    for i,i_df in enumerate(df_pand):
        try:
            # print(i,i_df.dtypes,i_df.columns) #i=0:4
            if add_label:
                i_df.columns=['index','query_id','query_title_id','label','probability']
            else:
                i_df.columns=['index','query_id','query_title_id','probability']
            i_df['prediction'] = [line[1] for line in i_df['probability']]
            i_df.drop('probability', axis=1,inplace=True)
            df_pand[i] = i_df
        except:
            continue
    df_pand = pd.concat(df_pand)
    df_pand.sort_values('index',ascending=True,inplace=True)
    df_pand.drop(['index'],axis=1,inplace=True)
    
    if add_label:
        df_pand.columns = ['query_id','query_title_id','label','prediction']
    else:
        df_pand.columns = ['query_id','query_title_id','prediction']
    return df_pand

In [23]:
pd.read_csv(open('./tmp_result_100000.csv','r'),nrows=5,header=None)

In [39]:
# # 合并结果
# import pandas as pd
# header_names=['query_id','title_id','prediction']

# file_paths = ['tmp_result_100000_0.csv',
#               'tmp_result_100000_1.csv',
#               'tmp_result_100000_2.csv',
#               'tmp_result_100000_3.csv',
#               'tmp_result_100000_4.csv']
# def res_merge(res1,res2):
#     result = pd.merge(res1, res2, on=['query_id', 'title_id'])
#     result['prediction']=result['prediction_x']+result['prediction_y']
#     result.drop(['prediction_x','prediction_y'],axis=1,inplace=True)
#     return result

# res_sum = pd.read_csv(open('./tmp_result_99999.csv','r'),nrows=None,header=None,names=header_names)
# for path in file_paths:
#     res_tmp = pd.read_csv(open(path,'r'),nrows=None,header=None,names=header_names)
#     res_sum = res_merge(res_sum, res_tmp)  # 求和
#     print("{} merge done.".format(path))


# res_sum['pred'] = res_sum['prediction']/(len(file_paths)+1)
# res_sum.drop(['prediction'],axis=1,inplace=True)

# res_sum

In [42]:
res_sum.to_csv('./result_0625_mean_400w.csv',header=None,index=0)
print("save done.")
pd.read_csv(open('./result_0625_mean_400w.csv','r'),nrows=100,header=None)

In [41]:
pd.read_csv(open('./result_0625_1104.csv','r'),nrows=5,header=None)

In [43]:
# result submit
!./kesci_submit -token b321e54849cd528c -file result_0625_mean_400w.csv